In [ ]:
import cv2
import numpy as np

img_dir = "data/"
img_list=["00125v.jpg","00149v.jpg","00153v.jpg","00351v.jpg","00398v.jpg","01112v.jpg"]

In [ ]:
def crop_img(img):
    h,w = img.shape
    for i in range(h):
        if np.mean(img[i,:]) < 210:
            break

    for j in range(h-1,0,-1):
        if np.mean(img[j,:]) < 210:
            break
    for k in range(w):
        if np.mean(img[:,k]) < 210:
            break
    for d in range(w-1,0,-1):
        if np.mean(img[:,d]) < 210:
            break
    return img[i:j,k:d].copy()

In [ ]:
def NCC(f,t):
    F=f-f.mean(axis=0)
    T=t-t.mean(axis=0)
    return np.sum((F/np.linalg.norm(F)) * (T/np.linalg.norm(T)))

def img_align(a,b,s_range):
    minn= -1
    ans = []
    for i in range(-s_range,s_range+1):
        for j in range(-s_range,s_range+1):
            result = NCC(a,np.roll(b,[i,j],axis=(0,1)))
            if result > minn:
                minn = result
                ans = [i,j]
    return ans

def SSD(img1,img2):
    a = np.sum((img1-img2)**2)
    # print(a)
    return a

def align_canny(a,b,s_range):
    c_a = cv2.GaussianBlur(a,(5,5),0)
    c_b = cv2.GaussianBlur(b,(5,5),0)
    c_a = cv2.Canny(c_a,100,150)
    c_b = cv2.Canny(c_b,100,150)
    minn= -1
    ans = []
    for i in range(-s_range,s_range+1):
        for j in range(-s_range,s_range+1):
            result = NCC(c_a,np.roll(c_b,[i,j],axis=(0,1)))
            if result > minn:
                minn = result
                ans = [i,j]
    return ans

In [ ]:
def downscale(img):
    result = cv2.pyrDown(img,(img.shape[0]//2,img.shape[1]//2))
    return result

def pyramid(img1,img2):
    pyra = [[img1,img2]]
    while pyra[-1][0].shape[1]> 400:
        pyra.append([downscale(pyra[-1][0]),downscale(pyra[-1][1])])
    
    offset = [0,0]
    
    for i in range(len(pyra)-1,0,-1):
        change = img_align(pyra[i][0],np.roll(pyra[i][1],offset,(0,1)),2)
        offset = list(np.add(offset,change))
        offset = [i*2 for i in offset]
    return offset
        
    

In [ ]:
img_dir = "data/"
img_list = ["00125v.jpg","00149v.jpg","00153v.jpg","00351v.jpg","00398v.jpg","01112v.jpg"]
img_dest = "result/"

for loc in img_list:
    img = cv2.imread(img_dir+loc,cv2.IMREAD_GRAYSCALE)
    img = crop_img(img)
    # img = cv2.GaussianBlur(img,(5,5),0)
    # img = cv2.Canny(img,200,300)
    h = int(img.shape[0]*1/3)
    blue = img[0:h,:]
    green = img[h:2*h,:]
    red = img[2*h:3*h,:]
    b2g = img_align(green,blue,15)
    r2g = img_align(green,red,15)
    print(loc,b2g, r2g)
    b_off=np.roll(blue,b2g,axis=(0,1))
    r_off=np.roll(red,r2g,axis=(0,1))
    c_img = (np.dstack((b_off,green,r_off))).astype(np.uint8)
    # cv2.imwrite(img_dest+loc,c_img)
    cv2.imshow("test",c_img)
    # cv2.imshow("test",img)
    cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
for loc in img_list:
    img = cv2.imread(dir+loc,cv2.IMREAD_GRAYSCALE)
    img = crop_img(img)
    h = int(img.shape[0]*1/3)
    blue = img[0:h,:]
    green = img[h:2*h,:]
    red = img[2*h:3*h,:]
    b2g = img_align(green,blue,15)
    r2g = img_align(green,red,15)
    print(b2g, r2g)
    b_off=np.roll(blue,b2g,axis=(0,1))
    r_off=np.roll(red,r2g,axis=(0,1))
    c_img = (np.dstack((b_off,green,r_off))).astype(np.uint8)
    cv2.imwrite(loc,c_img)

In [ ]:
dir_h = "data_hires/"
h_img_list = ["01047u", "01657u", "01861a"]
for loc in h_img_list:
    img = cv2.imread(dir_h+loc+".tif",cv2.IMREAD_GRAYSCALE)
    img = crop_img(img)
    h = int(img.shape[0]*1/3)
    blue = img[0:h,:]
    green = img[h:2*h,:]
    red = img[2*h:3*h,:]
    g2b = pyramid(blue,green)
    r2b = pyramid(blue,red)
    print(g2b, r2b)
    g_off=np.roll(green,g2b,axis=(0,1))
    r_off=np.roll(red,r2b,axis=(0,1))
    c_img = (np.dstack((blue,g_off,r_off))).astype(np.uint8)
    cv2.imwrite("result/"+loc+".jpg",c_img)


In [ ]:
# Test to see what channels should be selected for the lineup

dir_h = "data_hires/"
h_img_list = ["01047u.tif", "01657u.tif", "01861a.tif"]
for loc in h_img_list:
    img = cv2.imread(dir_h+loc,cv2.IMREAD_GRAYSCALE)
    img = crop_img(img)
    h = int(img.shape[0]*1/3)
    blue = img[0:h,:]
    green = img[h:2*h,:]
    red = img[2*h:3*h,:]
    for comb in [[blue,green,red,"blue"],[green,red,blue,"green"],[red,blue,green,"red"]]:
        a = pyramid(comb[0],comb[1])
        b = pyramid(comb[0],comb[2])
        a_off = np.roll(comb[1],a,axis=(0,1))
        b_off = np.roll(comb[2],b,axis=(0,1))
        if comb[3] == "blue":
            c_img = (np.dstack((blue,a_off,b_off))).astype(np.uint8)
        elif comb[3] == "green":
            c_img = (np.dstack((b_off,green,a_off))).astype(np.uint8)
        else:
            c_img = (np.dstack((a_off,b_off,red))).astype(np.uint8)
        cv2.imwrite("test/"+comb[3]+"_"+loc,c_img)


In [ ]:
def gamma(src, g):
    inv = 1 / g

    table = [((i / 255) ** inv) * 255 for i in range(256)]
    table = np.array(table, np.uint8)

    return cv2.LUT(src, table)

In [ ]:
def gray_world(img):
    b_image, g_image, r_image = cv2.split(img)
    mu = np.average(b_image)
    
    r_image = np.minimum(r_image*(mu /np.average(r_image)),255)
    g_image = np.minimum(g_image*(mu /np.average(g_image)),255)
    
    return (np.dstack((b_image,g_image,r_image))).astype(np.uint8)

In [ ]:
def equalize(img):
    b_image, g_image, r_image = cv2.split(img)

    b_image_eq = cv2.equalizeHist(b_image)
    g_image_eq = cv2.equalizeHist(g_image)
    r_image_eq = cv2.equalizeHist(r_image)
    
    return (np.dstack((b_image_eq,g_image_eq,r_image_eq))).astype(np.uint8)

In [ ]:
def auto_crop(img1):
    h,w,d = img.shape
    top = img[0:h//10,w//10:-(w//10),:]
    down = img[-(h//10):,w//10:-(w//10),:]
    left = img[h//10:-(h//10),0:w//10,:]
    right = img[h//10:-(h//10),-(w//10):w,:]
    crop_top,crop_down,crop_left,crop_right = 0,h,0,w
    for part_img,part in zip([top,down,left,right],["top","down","left","right"]):
        lines = cv2.HoughLinesP(cv2.Canny(part_img,100,150),rho = 1, theta = np.pi/180,threshold = 150,minLineLength = 100,maxLineGap = 20)
        if lines is None:
            if part == "down":
                crop_down = 0
            elif part == "right":
                crop_right = 0
            else:
                pass
            print("img has no lines")
            continue
        for line in lines:
            # for x1,y1,x2,y2 in line:
            #     # cv2.line(part_img,(x1,y1),(x2,y2),(255,0,255),1)
            #     print(part,line)
            if part in ["left","right"]:
                for x1,_,x2,_ in line:
                    if (x1 - x2)**2 >= 900:
                        continue
                    if part == "left":
                        crop_left = max([crop_left,x1,x2])
                    else:
                        crop_right = min([crop_right,x1,x2])
            else:
                for _,y1,_,y2 in line:
                    if (y1-y2)**2 >= 900:
                        continue
                    if part == "top":
                        crop_top = max([crop_top,y1,y2])
                    else:
                        crop_down = min([crop_down,y1,y2])
    # print("NEW")
    # print(img.shape)
    # print(crop_top,crop_down,crop_left,crop_right)    
    test_img = img[crop_top:h-crop_down,crop_left:w-crop_right,:]
    return test_img
    # cv2.imshow("test",test_img)
    # cv2.imshow("orig",img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
        

In [ ]:
img_list = ["00125v.jpg","00149v.jpg","00153v.jpg","00351v.jpg","00398v.jpg","01112v.jpg","01047u.jpg", "01657u.jpg", "01861a.jpg"]
# img_list = ["01047u.tif", "01657u.tif", "01861a.tif"]
for loc in img_list:
    print("result/"+loc)
    img = cv2.imread("result/"+loc)
    result = auto_crop(img)
    cv2.imwrite("result/crop_"+loc,result)
# img = cv2.imread("00149v.jpg")
# auto_crop(img)

In [ ]:
# Testing equalize White balance and gamma correction
img_list = ["00125v.jpg","00149v.jpg","00153v.jpg","00351v.jpg","00398v.jpg","01112v.jpg"]
for loc in img_list:
    img = cv2.imread("test/"+loc)
    # g_img = equalize(img)
    # g_img = gray_world(g_img)
    # g_img = cv2.GaussianBlur(img,(5,5),0)
    # g_img = cv2.Canny(img,100,200)
    g_img = gamma(g_img,1.3)
    
    cv2.imshow("out",g_img)
    cv2.imshow("ori",img)
    cv2.waitKey(0)
cv2.destroyAllWindows()
